In [ ]:
import pandas as pd
from openpyxl import Workbook
from openpyxl.chart import ScatterChart, Series, Reference
from openpyxl.utils.dataframe import dataframe_to_rows

# Create a sample dataset
data = {
    "quantity_sqyd": [1000, 3000, 5000, 8000, 12000, 18000, 25000, 33000, 40000, 60000],
    "vendor_estimated_cost": [200, 600, 1000, 1200, 1500, 1900, 2300, 2700, 3100, 3500],
    "actual_freight_cost": [180, 590, 1100, 1300, 1600, 1850, 2500, 2900, 3300, 5100],
    "ftl_cost": [1350] * 10,
    "ltl_only_cost": [190, 610, 950, 1150, None, 2000, None, 2800, 3500, 4300],
    "optimal_cost": [180, 580, 920, 1100, 1400, 1700, 2000, 2400, 2700, 3100],
}
df = pd.DataFrame(data)

# Create workbook and worksheet
wb = Workbook()
ws = wb.active
ws.title = "Freight Chart"

# Add data to worksheet
for r in dataframe_to_rows(df, index=False, header=True):
    ws.append(r)

# Set up chart
chart = ScatterChart()
chart.title = "Freight Cost Overlay"
chart.x_axis.title = "Quantity (SQYD)"
chart.y_axis.title = "Freight Cost ($)"
chart.legend.position = "r"

x_values = Reference(ws, min_col=1, min_row=2, max_row=11)

# Add each series
def add_series(title, y_col, marker=None):
    y_values = Reference(ws, min_col=y_col, min_row=2, max_row=11)
    series = Series(y_values, x_values, title=title)
    if marker:
        series.marker.symbol = marker
        series.graphicalProperties.line.noFill = True
    chart.series.append(series)

add_series("Vendor Cost Curve", 2)
add_series("Actual Invoice Freight", 3, marker="circle")
add_series("FTL Cost", 4)
add_series("LTL Only Cost", 5, marker="diamond")
add_series("Optimal Cost", 6, marker="square")

# Add chart to sheet
ws.add_chart(chart, "H2")

# Save to file
wb.save("freight_chart_notebook.xlsx")